In [1]:
from catboost import CatBoostRegressor, Pool
import pandas as pd
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt
from sklearn import metrics
import numpy as np
import pickle

pd.options.display.max_columns = 200

In [2]:
def read_file(path):
    df = pd.read_excel(path)
    df.columns = df.iloc[4]
    df = df.iloc[5:][df.columns[:-1]]

    columns = df.columns.tolist()

    columns[3] = 'продажи'
    columns[16] = 'заболеваемость'
    columns[33] = 'ТВ реклама, руб'
    columns[35] = 'интернет реклама, руб'
    columns[51] = 'wordstat'

    df.columns = columns
    df.loc[df['неделя']==53, 'неделя'] = 52

    df['Начало нед'] = pd.to_datetime(df['Начало нед'])
    df['ТВ реклама, руб'] = df['ТВ реклама, руб'].replace(' ', 0).astype(float)

    return df

df = read_file('data/train.xlsx')
df.loc[df['неделя']==53, 'неделя'] = 52

/var/folders/bb/jzxxldrn52l0stzqmszjx6840000gn/T/ipykernel_39042/695145740.py:18: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['ТВ реклама, руб'] = df['ТВ реклама, руб'].replace(' ', 0).astype(float)


In [3]:
def create_features(df):

    week_info = df.groupby('неделя')['продажи'].agg(['sum', 'count'])
    week_info.columns = [f'week_{x}' for x in week_info.columns]

    df[f'target'] = df['продажи'] / df['продажи'].shift(1)

    df['feature_illnesses'] = df['заболеваемость'].shift(1)

    df['feature_last_5_wordstat'] = df['wordstat'].shift(1).rolling(window=5).mean()
    df['feature_last_15_wordstat'] = df['wordstat'].shift(1).rolling(window=15).mean()

    df['feature_last_5_adv'] = df['ТВ реклама, руб'].shift(1).rolling(window=5).mean()
    df['feature_last_15_adv'] = df['ТВ реклама, руб'].shift(1).rolling(window=15).mean()

    df['feature_last_5_internet_adv'] = df['интернет реклама, руб'].shift(1).rolling(window=5).mean()
    df['feature_last_15_internet__adv'] = df['интернет реклама, руб'].shift(1).rolling(window=15).mean()

    df['feature_week'] = df['неделя']

    for i in range(1, 16, 4):
        df[f'feature_month_diff_{i}'] = df['продажи'].shift(i) / df['продажи'].shift(i+4)

    for i in [7, 13, 20]:
        df[f'feature_big_diff_{i}_weeks'] = df['продажи'].shift(1) / df['продажи'].shift(i)

    bad_cols = [col for col in df.columns if 'feature' not in col and 'target' not in col]
    df.drop(bad_cols, axis=1, inplace=True)

    feature_names = {'feature_illnesses': 'кол-во больных на последней неделе',
 'feature_last_5_wordstat': 'среднее кол-во просмотров wordstat за 5 недель',
 'feature_last_15_wordstat': 'среднее кол-во просмотров wordstat за 15 недель',
 'feature_last_5_adv': 'средние затраты на тв рекламу последние 5 недель',
 'feature_last_15_adv': 'средние затраты на тв рекламу последние 15 недель',
 'feature_last_5_internet_adv': 'средние затраты на интернет рекламу последние 5 недель',
 'feature_last_15_internet__adv': 'средние затраты на интернет рекламу последние 15 недель',
 'feature_week': 'номер недели',
 'feature_month_diff_1': 'изменение продаж месяц назад',
 'feature_month_diff_5': 'изменение продаж 2 месяца назад',
 'feature_month_diff_9': 'изменение продаж 3 месяца назад',
 'feature_month_diff_13': 'изменение продаж 4 месяца назад',
 'feature_big_diff_7_weeks': 'изменение продаж за последние 2 месяца',
 'feature_big_diff_13_weeks': 'изменение продаж за последние 3 месяца',
 'feature_big_diff_20_weeks': 'изменение продаж за последние 4 месяца'}
    
    df.rename(feature_names, axis=1, inplace=True)
    
    return df

df = create_features(df)

In [4]:
df = df[df['target'].notna()].reset_index(drop=True)
df = df.loc[20:].reset_index(drop=True)

In [7]:
test_size = 0.4

all_models = []
all_metrics = []
all_preds = []

cur_df = df[(df[f'target'].notna())&(df['средние затраты на тв рекламу последние 5 недель'] > 0)]

train_data, valid_data = cur_df[:int(len(cur_df) * (1-test_size))], cur_df[int(len(cur_df) * (1-test_size)):]

X_train, y_train = train_data.drop([x for x in train_data.columns if 'target' in x], axis=1), train_data[f'target']
X_valid, y_valid = valid_data.drop([x for x in valid_data.columns if 'target' in x], axis=1), valid_data[f'target']


model = CatBoostRegressor(
    depth=2,
    learning_rate=0.03,
    iterations=3000,
    loss_function='MAPE',
    eval_metric='R2',
    custom_metric=['R2'],

    random_state=42,
    thread_count=5
)

model.fit(X_train, y_train, eval_set=(X_valid, y_valid), verbose=1000)
all_models.append(model)

preds = model.predict(X_valid)
all_preds.append(preds)

r2_metric = metrics.r2_score(y_valid, preds)
mape_metric = metrics.mean_absolute_percentage_error(y_valid, preds)
all_metrics.append([r2_metric, mape_metric])

print(f'R2: {round(r2_metric, 5)}')
print(f'MAPE: {round(mape_metric, 5)}')

print()

0:	learn: -0.0000360	test: 0.0015751	best: 0.0015751 (0)	total: 55.5ms	remaining: 2m 46s
1000:	learn: 0.1967136	test: 0.1167971	best: 0.1570845 (147)	total: 130ms	remaining: 260ms
2000:	learn: 0.2238778	test: 0.1066156	best: 0.1570845 (147)	total: 206ms	remaining: 103ms
2999:	learn: 0.2301255	test: 0.1092143	best: 0.1570845 (147)	total: 282ms	remaining: 0us

bestTest = 0.1570845081
bestIteration = 147

Shrink model to first 148 iterations.
R2: 0.15708
MAPE: 0.0688



In [8]:
model.save_model("info_model.cbm")

In [9]:
pd.DataFrame({
    'name': X_train.columns,
    'imp': model.get_feature_importance()
}).sort_values('imp', ascending=False)

,name,imp
1,среднее кол-во просмотров wordstat за 5 недель,26.012261
2,среднее кол-во просмотров wordstat за 15 недель,13.925666
8,изменение продаж месяц назад,10.572021
12,изменение продаж за последние 2 месяца,10.478944
9,изменение продаж 2 месяца назад,9.351804
11,изменение продаж 4 месяца назад,6.341930
14,изменение продаж за последние 4 месяца,4.840989
10,изменение продаж 3 месяца назад,3.940789
7,номер недели,3.627323
0,кол-во больных на последней неделе,3.073558


In [10]:
print(f'R2: {round(np.mean([x[0] for x in all_metrics]), 5)}')
print(f'MAPE: {round(np.mean([x[1] for x in all_metrics]), 5)}')

R2: 0.15708
MAPE: 0.0688


R2: 0.42683
MAPE: 0.10619

-----

In [13]:
from catboost import CatBoostRegressor, Pool
import pandas as pd
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt
from sklearn import metrics
import numpy as np
import pickle

pd.options.display.max_columns = 200

In [14]:
model = CatBoostRegressor()
model.load_model('info_model.cbm')

In [15]:
def read_file(path):
    df = pd.read_excel(path)
    df.columns = df.iloc[4]
    df = df.iloc[5:][df.columns[:-1]]

    columns = df.columns.tolist()

    columns[3] = 'продажи'
    columns[16] = 'заболеваемость'
    columns[33] = 'ТВ реклама, руб'
    columns[35] = 'интернет реклама, руб'
    columns[51] = 'wordstat'
    
    df.columns = columns
    df.loc[df['неделя']==53, 'неделя'] = 52
    df['ТВ реклама, руб'] = df['ТВ реклама, руб'].replace(' ', 0).astype(float)

    return df

df = read_file('data/train.xlsx')
df.loc[df['неделя']==53, 'неделя'] = 52

/var/folders/bb/jzxxldrn52l0stzqmszjx6840000gn/T/ipykernel_39042/2888774503.py:16: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['ТВ реклама, руб'] = df['ТВ реклама, руб'].replace(' ', 0).astype(float)


In [16]:
def create_features(df):

    week_info = df.groupby('неделя')['продажи'].agg(['sum', 'count'])
    week_info.columns = [f'week_{x}' for x in week_info.columns]

    df[f'target'] = df['продажи'] / df['продажи'].shift(1)

    df['feature_illnesses'] = df['заболеваемость'].shift(1)

    df['feature_last_5_wordstat'] = df['wordstat'].shift(1).rolling(window=5).mean()
    df['feature_last_15_wordstat'] = df['wordstat'].shift(1).rolling(window=15).mean()

    df['feature_last_5_adv'] = df['ТВ реклама, руб'].shift(1).rolling(window=5).mean()
    df['feature_last_15_adv'] = df['ТВ реклама, руб'].shift(1).rolling(window=15).mean()

    df['feature_last_5_internet_adv'] = df['интернет реклама, руб'].shift(1).rolling(window=5).mean()
    df['feature_last_15_internet__adv'] = df['интернет реклама, руб'].shift(1).rolling(window=15).mean()

    df['feature_week'] = df['неделя']

    for i in range(1, 16, 4):
        df[f'feature_month_diff_{i}'] = df['продажи'].shift(i) / df['продажи'].shift(i+4)

    for i in [7, 13, 20]:
        df[f'feature_big_diff_{i}_weeks'] = df['продажи'].shift(1) / df['продажи'].shift(i)

    bad_cols = [col for col in df.columns if 'feature' not in col and 'target' not in col]
    df.drop(bad_cols, axis=1, inplace=True)

    feature_names = {'feature_illnesses': 'кол-во больных на последней неделе',
 'feature_last_5_wordstat': 'среднее кол-во просмотров wordstat за 5 недель',
 'feature_last_15_wordstat': 'среднее кол-во просмотров wordstat за 15 недель',
 'feature_last_5_adv': 'средние затраты на тв рекламу последние 5 недель',
 'feature_last_15_adv': 'средние затраты на тв рекламу последние 15 недель',
 'feature_last_5_internet_adv': 'средние затраты на интернет рекламу последние 5 недель',
 'feature_last_15_internet__adv': 'средние затраты на интернет рекламу последние 15 недель',
 'feature_week': 'номер недели',
 'feature_month_diff_1': 'изменение продаж месяц назад',
 'feature_month_diff_5': 'изменение продаж 2 месяца назад',
 'feature_month_diff_9': 'изменение продаж 3 месяца назад',
 'feature_month_diff_13': 'изменение продаж 4 месяца назад',
 'feature_big_diff_7_weeks': 'изменение продаж за последние 2 месяца',
 'feature_big_diff_13_weeks': 'изменение продаж за последние 3 месяца',
 'feature_big_diff_20_weeks': 'изменение продаж за последние 4 месяца'}
    
    df.rename(feature_names, axis=1, inplace=True)
    
    return df

df = create_features(df)
df = df.reset_index(drop=True)

In [17]:
df = df[df['target'].notna()].dropna()

In [18]:
model.predict(df[model.feature_names_])

array([0.95205372, 1.00725394, 1.02685994, 1.03539017, 1.01890825,
       1.03141576, 1.0100576 , 1.01934342, 1.0082939 , 1.01037523,
       1.01873457, 1.03292876, 1.03092404, 1.03212255, 1.01367852,
       1.00757551, 1.00680974, 0.99810615, 0.95648003, 0.98742892,
       0.99153844, 0.97459671, 0.95801217, 0.9741478 , 0.98817747,
       1.01568095, 1.00771577, 1.01445973, 1.00529094, 1.01499466,
       1.00570978, 1.00831716, 0.99840064, 1.02788673, 1.02412729,
       1.02903332, 1.02735591, 1.02703558, 0.973673  , 0.97397358,
       0.98104099, 0.99852321, 0.99390521, 0.98192917, 0.87888061,
       0.81337751, 0.77906171, 0.88323969, 0.87249378, 0.88473256,
       0.89631455, 0.944136  , 0.95771854, 0.96942595, 0.97187184,
       0.98314402, 1.00507284, 1.01543259, 1.02589705, 1.04721597,
       1.0432216 , 1.03918052, 1.02773495, 1.02841886, 1.02628349,
       1.03169467, 1.0294182 , 1.00317065, 1.00848211, 0.9999699 ,
       0.96402048, 0.99144542, 0.99667582, 0.9958277 , 0.98896